In [1]:
# Import requisite classes
import niworkflows.interfaces.report_base as nrc
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.base import traits
from nipype.interfaces.mixins import reporting
from nipype.interfaces.base import DynamicTraitedSpec
from nipype.interfaces.base import File
from nipype.interfaces.base import BaseInterfaceInputSpec

In [2]:
import pdb

In [ ]:
# %load niviz/interfaces/views.py
# emacs: -*- mode: python; py-indent-offset: 4; indent-tabs-mode: nil -*-
# vi: set ft=python sts=4 ts=4 sw=4 et:

import niworkflows.interfaces.report_base as nrc
from nipype.interfaces.base import File
from nipype.interfaces.mixins import reporting
"""
ReportCapable concrete classes for generating reports as side-effects
"""


class _IRegInputSpecRPT(nrc._SVGReportCapableInputSpec):

    bg_nii = File(exists=True,
                  usedefault=False,
                  resolve=True,
                  desc='Background NIFTI for SVG',
                  mandatory=True)

    fg_nii = File(exists=True,
                  usedefault=False,
                  resolve=True,
                  desc='Foreground NIFTI for SVG',
                  mandatory=True)

    contours = File(exists=True,
                    usedefault=False,
                    resolve=True,
                    desc='Contours to include in image',
                    mandatory=True)


class _IRegOutputSpecRPT(reporting.ReportCapableOutputSpec):
    pass


class IRegRPT(nrc.RegistrationRC):

    input_spec = _IRegInputSpecRPT
    output_spec = _IRegOutputSpecRPT

    def _post_run_hook(self, runtime):

        self._fixed_image = self.inputs.bg_nii
        self._moving_image = self.inputs.fg_nii

        return super(IRegRPT, self)._post_run_hook(runtime)

    def _run_interface(self, runtime):
        return runtime


In [3]:
class _IRegInputSpecRPT(nrc._SVGReportCapableInputSpec,
                       BaseInterfaceInputSpec):
    
    '''
    Input specification for IRegRPT, implements:
    
    bg_nii: Input background NIFTI image
    fg_nii: Input foreground NIFTI image
    contour: Optional outlining NIFTI Image
    
    Bases _SVGReportCapableInputSpec which implements:
    
    out_report: Filename trait
    compress_report: ["auto", true, false]
    
    '''
    bg_nii = File(exists=True,
                  usedefault=False,
                  resolve=True,
                  desc='Background image of SVG',
                  mandatory=True)
    
    fg_nii = File(exists=True,
                  usedefault=False,
                  resolve=True,
                  desc='Foreground image of SVG',
                  mandatory=True)
    
    contour = File(exists=True,
                  udedefault=False,
                  resolve=True,
                  desc='Contours to include in image',
                  mandatory=False)

In [4]:
class _IRegOutputSpecRPT(reporting.ReportCapableOutputSpec):
    pass

In [5]:
class IRegRPT(nrc.RegistrationRC):
    '''
    Class to generate registration images from pre-existing
    NIFTI files. 
    
    Effectively acts as an Identity node with report
    generation as a side-effect.
    '''

    # Use our declared IO specs
    input_spec = _IRegInputSpecRPT
    output_spec = _IRegOutputSpecRPT

    
    def _post_run_hook(self, runtime):
        
        '''
        Do nothing but propogate properties
        to (first) parent class of IRegRPT
        that is nrc.RegistrationRC
        '''
        
        # Set variables for `nrc.RegistrationRC`
        self._fixed_image = self.inputs.bg_nii
        self._moving_image = self.inputs.fg_nii
        
        # Propogate to RegistrationRC superclass
        return super(IRegRPT, self)._post_run_hook(runtime)

    
    def _run_interface(self, runtime):
        return runtime

In [10]:
# Set up inputs
pre='./data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_desc-preproc_T1w.nii.gz'
post='./data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_desc-aseg_dseg.nii.gz'

In [11]:
# Set up input spec and generate report as side-effect
regrpt = IRegRPT(generate_report=True)
regrpt.inputs.bg_nii = pre
regrpt.inputs.fg_nii = post
regrpt.inputs.out_report = './tmp/test.svg'

In [13]:
# Run no actual computation, but generate report as side-effect
regrpt.run()

# This should work successfully!

/scratch/bselby/.pyenv/versions/3.8.0/lib/python3.8/site-packages/nilearn/image/resampling.py:512: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/scratch/bselby/.pyenv/versions/3.8.0/lib/python3.8/site-packages/matplotlib/image.py:491: RuntimeWarning: overflow encountered in true_divide
  vrange /= ((a_max - a_min) / frac)
/scratch/bselby/.pyenv/versions/3.8.0/lib/python3.8/site-packages/nilearn/image/resampling.py:512: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/scratch/bselby/.pyenv/versions/3.8.0/lib/python3.8/site-packages/nilearn/image/resampling.py:512: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


In [52]:
from nipype.interfaces.base import InputMultiPath

class _ISegInputSpecRPT(nrc._SVGReportCapableInputSpec,
                       BaseInterfaceInputSpec):
    '''
    Input specification for ISegRPT, implements:
    
    anat_file: Input anatomical image
    seg_files: Input segmentation image(s) - can be a list or a single file
    mask_file: Input ROI mask
        
    Bases _SVGReportCapableInputSpec which implements:
    
    out_report: Filename trait
    compress_report: ["auto", true, false]
    
    '''
    anat_file  = File(exists=True,
                  usedefault=False,
                  resolve=True,
                  desc='Anatomical image of SVG',
                  mandatory=True)
    
    seg_files = InputMultiPath( 
                    File(exists=True,
                         usedefault=False,
                         resolve=True),
                    desc='Segmentation image of SVG',     
                    mandatory=True)
    
    mask_file = File(exists=True,
                  udedefault=False,
                  resolve=True,
                  desc='ROI Mask for mosaic',
                  mandatory=True)
    

class _ISegOutputSpecRPT(reporting.ReportCapableOutputSpec):
    pass

class ISegRPT(nrc.SegmentationRC):
    '''
    Class to generate registration images from pre-existing
    NIFTI files. 
    
    Effectively acts as an Identity node with report
    generation as a side-effect.
    '''

    # Use our declared IO specs
    input_spec = _ISegInputSpecRPT
    output_spec = _ISegOutputSpecRPT
    
    
    
    def _post_run_hook(self, runtime):
        
        '''
        Do nothing but propogate properties
        to (first) parent class of ISegRPT
        that is nrc.SegmentationRC
        '''
        
        # Set variables for `nrc.SegmentationRC`
        self._anat_file = self.inputs.anat_file
        self._seg_files = self.inputs.seg_files
        self._mask_file = self.inputs.mask_file
        self._masked = False if self.inputs.mask_file is None else True
                
        # Propogate to RegistrationRC superclass
        return super(ISegRPT, self)._post_run_hook(runtime)
    


    def _run_interface(self, runtime):
        return runtime

In [10]:
fsl.FAST.help()

Wraps the executable command ``fast``.

FSL FAST wrapper for segmentation and bias correction

For complete details, see the `FAST Documentation.
<https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FAST>`_

Examples
--------
>>> from nipype.interfaces import fsl
>>> fastr = fsl.FAST()
>>> fastr.inputs.in_files = 'structural.nii'
>>> fastr.inputs.out_basename = 'fast_'
>>> fastr.cmdline
'fast -o fast_ -S 1 structural.nii'
>>> out = fastr.run()  # doctest: +SKIP

Inputs::

        [Mandatory]
        in_files: (a list of items which are a pathlike object or string
                  representing an existing file)
                image, or multi-channel set of images, to be segmented
                argument: ``%s``, position: -1

        [Optional]
        out_basename: (a pathlike object or string representing a file)
                base name of output files
                argument: ``-o %s``
        number_classes: (1 <= an integer <= 10)
                number of tissue-type classes
            

In [11]:
from nipype.interfaces import fsl
from nipype.testing import anatfile

fastr = fsl.FAST()

fastr.inputs.in_files = './data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'
# Seg file:
# fastr.inputs.in_files = './data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_dseg.nii.gz'
fastr.out_basename = './tmp/fast_'
out = fastr.run()

201216-16:59:35,927 nipype.interface INFO:
	 stderr 2020-12-16T16:59:35.927322:terminate called after throwing an instance of 'NiftiException'
201216-16:59:35,930 nipype.interface INFO:
	 stderr 2020-12-16T16:59:35.927322:  what():  Error: cant open file data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_space-MNI152NLin2009cAsym_desc-preproc_T1w_seg.nii.gz
201216-16:59:36,39 nipype.interface INFO:
	 stderr 2020-12-16T16:59:36.039709:Aborted


RuntimeError: Command:
fast -S 1 data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz
Standard output:

Standard error:
terminate called after throwing an instance of 'NiftiException'
  what():  Error: cant open file data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_space-MNI152NLin2009cAsym_desc-preproc_T1w_seg.nii.gz
Aborted
Return code: 134

In [ ]:
plot_anat('/output/T1w_nipype_bet.nii.gz', title='original',
          display_mode='ortho', dim=-1, draw_cross=False, annotate=False);

In [53]:
# Set up segmentation inputs
anat_file = './data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_desc-preproc_T1w.nii.gz'
seg_file  = './data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_desc-aseg_dseg.nii.gz'
mask_file = './data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_desc-brain_mask.nii.gz' 
mask_file = None

In [55]:
# Set up input spec and generate report as side-effect
segrpt = ISegRPT(generate_report=True)
segrpt.inputs.anat_file = anat_file
segrpt.inputs.seg_files = seg_file
# segrpt.inputs.mask_file = mask_file
segrpt.inputs.out_report = './tmp/seg_out.svg'

In [56]:
# Run no actual computation, but generate report as side-effect
segrpt.run()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
segrpt._masked